In [0]:
%pip install -r requirements.txt

In [0]:
catalog = "serverless_lh10u9_catalog"
schema = "pixels_solacc"
table = "object_catalog"
REGISTERED_MODEL_NAME = f"{catalog}.{schema}.dicomweb-serving"

In [0]:
"""
Log the DICOMweb PyFunc model to MLflow.

Run inside a Databricks notebook (``%run ./log_model``) or as a script::

    python log_model.py

The script:

1. Instantiates :class:`DICOMwebServingModel`.
2. Points at the ``dicom_web/`` source tree as a model artifact
   (handlers, wrapper, SQL client, caches — everything the serving
   endpoint needs at runtime).
3. Logs the model with an explicit signature and pip requirements.
4. Optionally registers it in the MLflow Model Registry (uncomment
   the last section).
"""

import os
from pathlib import Path

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import ColSpec, Schema

from dicomweb_pyfunc import DICOMwebServingModel

# ---------------------------------------------------------------------------
# Paths
# ---------------------------------------------------------------------------

_this_dir = Path(os.getcwd())
_dicom_web_source = _this_dir.parent / "dicom_web"

print(_dicom_web_source)

if not _dicom_web_source.is_dir():
    raise FileNotFoundError(
        f"DICOMweb source directory not found at {_dicom_web_source}. "
        "Make sure this script is located in "
        "dbx/pixels/resources/dicom_web_model/ alongside the dicom_web/ folder."
    )

# ---------------------------------------------------------------------------
# Model signature
# ---------------------------------------------------------------------------

input_schema = Schema([
    ColSpec("string", "method"),
    ColSpec("string", "path"),
    ColSpec("string", "query_string"),
    ColSpec("string", "headers"),
    ColSpec("string", "body"),
    ColSpec("string", "body_encoding"),
])
output_schema = Schema([ColSpec("string")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

# ---------------------------------------------------------------------------
# Pip requirements (mirrors dicom_web/requirements.txt + httpx for ASGI)
# ---------------------------------------------------------------------------

pip_requirements = [
    "databricks-pixels @ git+https://github.com/databricks-industry-solutions/pixels@features/dicom_web_integration",
    "databricks-sql-connector>=3.0.0",
    "databricks-sdk",
    "fastapi",
    "uvicorn",
    "httpx",
    "requests-toolbelt",
    "zstd",
    "psycopg2-binary",
    "psutil>=5.9.0",
    "python-multipart",
    "pydicom>=2.4.0",
    "pylibjpeg",
    "pylibjpeg-libjpeg",
    "pylibjpeg-openjpeg",
]

In [0]:
# ---------------------------------------------------------------------------
# Log model
# ---------------------------------------------------------------------------

with mlflow.start_run(run_name="dicomweb-serving-model") as run:
    model_info = mlflow.pyfunc.log_model(
        artifact_path="dicomweb_model",
        python_model=DICOMwebServingModel(),
        artifacts={ "dicom_web_source": str(_dicom_web_source), },
        pip_requirements=pip_requirements,
        signature=signature,
        code_paths=[str(_this_dir / "dicomweb_pyfunc.py")],
    )

    print(f"Model logged:  {model_info.model_uri}")
    print(f"Run ID:        {run.info.run_id}")

# ---------------------------------------------------------------------------
# (Optional) Register the model — uncomment to auto-register after logging
# ---------------------------------------------------------------------------
mlflow.register_model(model_info.model_uri, REGISTERED_MODEL_NAME)
print(f"Registered as: {REGISTERED_MODEL_NAME}")

In [0]:
cookie_header = {"Cookie": "pixels_table=serverless_lh10u9_catalog.pixels_solacc.object_catalog"}

sample_inputs = [
    {
        "method": "GET",
        "path": "/api/dicomweb/studies/1.2.840.113619.2.55.3.604688435.781.159785.1.1",
        "query_string": "",
        "headers": '{"Accept": "application/dicom+json", "Cookie": "pixels_table=serverless_lh10u9_catalog.pixels_solacc.object_catalog"}',
        "body": "",
        "body_encoding": "utf-8"
    },
    {
        "method": "GET",
        "path": "/api/dicomweb/studies",
        "query_string": "",
        "headers": '{"Accept": "application/dicom+json", "Cookie": "pixels_table=serverless_lh10u9_catalog.pixels_solacc.object_catalog"}',
        "body": "",
        "body_encoding": "utf-8"
    },
    {
        "method": "POST",
        "path": "/api/dicomweb/studies",
        "query_string": "",
        "headers": '{"Content-Type": "application/dicom", "Cookie": "pixels_table=serverless_lh10u9_catalog.pixels_solacc.object_catalog"}',
        "body": "<binary DICOM data>",
        "body_encoding": "base64"
    }
]


In [0]:
import base64
import json
from mlflow.pyfunc import PythonModelContext
import os
import zstd

os.environ['DATABRICKS_PIXELS_TABLE'] = 'serverless_lh10u9_catalog.pixels_solacc.object_catalog'
os.environ['DATABRICKS_WAREHOUSE_ID'] = '8f5ac1fda8c18b52'

context: PythonModelContext = PythonModelContext(artifacts={"dicom_web_source": str(_dicom_web_source)}, model_config={})
model = DICOMwebServingModel()
model.load_context(context)
result = model.predict(context, model_input=sample_inputs[1])

print(zstd.ZSTD_uncompress(base64.b64decode(json.loads(result)['body'])))

In [0]:
from dbx.pixels.m2m import DatabricksM2MAuth
from databricks.sdk import WorkspaceClient

# Define scope and key names for the credentials

scope_name = "pixels_scope"
sp_name = "pixels_sp"

sp_id_key = "pixels_sp_id"
sp_app_id_key = "pixels_sp_app_id"
sp_secret_key = "pixels_sp_secret"
token_key = "pixels_token"

table_name = f"{catalog}.{schema}.{table}"
volume = f"{catalog}.{schema}.pixels_volume"

m2m_client = None

delete_all_sercrets = False

try:
        m2m_client = DatabricksM2MAuth(
            principal_name="pixels_sp",
            account_api_token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get(),
            secrets_scope_name=scope_name,
            secrets_client_id_key=sp_id_key,
            secrets_client_app_id_key=sp_app_id_key,
            secrets_client_secret_key=sp_secret_key,
            workspace_url=WorkspaceClient().config.host,
        )
        m2m_client.grant_permissions(table_name, volume)
except Exception as e:
        print(e)